In [1]:
#r "nuget:Microsoft.ML,1.5.1"
#r "nuget:Microsoft.ML.FastTree,0.17.1"

In [1]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

In [1]:
class ModelInput
{
    [ColumnName("Need"), LoadColumn(0)]
    public string Need { get; set; }


    [ColumnName("Percentage"), LoadColumn(1)]
    public float Percentage { get; set; }


    [ColumnName("Priority"), LoadColumn(2)]
    public string Priority { get; set; }
}

public class ModelOutput
{
    // ColumnName attribute is used to change the column name from
    // its default value, which is the name of the field.
    [ColumnName("PredictedLabel")]
    public String Prediction { get; set; }
    public float[] Score { get; set; }
}

In [1]:
IEstimator<ITransformer> BuildTrainingPipeline(MLContext context)
{
    // Data process configuration with pipeline data transformations 
    var dataProcessPipeline = context.Transforms.Conversion.MapValueToKey("Priority", "Priority")
                                .Append(context.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair("Need", "Need") }))
                                .Append(context.Transforms.Concatenate("Features", new[] { "Need", "Percentage" }))
                                .AppendCacheCheckpoint(context);
    // Set the training algorithm 
    var trainer = context.MulticlassClassification.Trainers.OneVersusAll(context.BinaryClassification.Trainers.FastTree(labelColumnName: "Priority", featureColumnName: "Features"), labelColumnName: "Priority")
                                .Append(context.Transforms.Conversion.MapKeyToValue("PredictedLabel",  "PredictedLabel"));

    var trainingPipeline = dataProcessPipeline.Append(trainer);

    return trainingPipeline;
}

In [1]:
ITransformer TrainModel(MLContext context, IDataView trainingDataView, IEstimator<ITransformer> trainingPipeline)
{
    Console.WriteLine("=============== Training  model ===============");

    var model = trainingPipeline.Fit(trainingDataView);

    Console.WriteLine("=============== End of training process ===============");
    return model;
}

In [1]:
void SaveModel(MLContext mlContext, ITransformer mlModel, string modelRelativePath, DataViewSchema modelInputSchema)
{
    var modelPath = Path.Combine("models", modelRelativePath);
    var path = Path.GetFullPath(modelPath);
    // Save/persist the trained model to a .ZIP file
    Console.WriteLine($"=============== Saving the model  ===============");
    mlContext.Model.Save(mlModel, modelInputSchema, path);
    Console.WriteLine("The model is saved to {0}", path);
}

In [1]:
PredictionEngine<ModelInput, ModelOutput> CreatePredictionEngine()
{
    var modelPath = Path.Combine("models", "Servo.zip");
    var MLNetModelPath = Path.GetFullPath(modelPath);

    // Create new MLContext
    MLContext mlContext = new MLContext();

    // Load model & create prediction engine
    var mlModel = mlContext.Model.Load(MLNetModelPath, out var modelInputSchema);
    var predEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(mlModel);

    return predEngine;
}

In [1]:
void Predictions(ModelInput[] inputs)
{
    foreach (var input in inputs)
    {
        var engine = new Lazy<PredictionEngine<ModelInput, ModelOutput>>(CreatePredictionEngine);
        var result = engine.Value.Predict(input);
        var priority = false;
        
        switch (result.Prediction)
        {
            default:
            case "FALSE":
                priority = false;
                break;
            case "TRUE":
                priority = true;
                break;
        }
        
        Console.WriteLine($"Need: {input.Need}{Environment.NewLine}" + 
        $"Percentage: {input.Percentage}{Environment.NewLine}" + 
        $"Predicted Priority value {priority}{Environment.NewLine}" + 
        $"Predicted Priority scores: [{string.Join(",", result.Score)}]");
    }
}

In [1]:
var sampleData = new ModelInput[]
{
    new ModelInput
    {
        Need = "Social",
        Percentage = 27F,
    },
    new ModelInput
    {
        Need = "Bladder",
        Percentage = 76F,
    },
    new ModelInput
    {
        Need = "Energy",
        Percentage = 55F,
    },
    new ModelInput
    {
        Need = "Social",
        Percentage = 20F,
    },
    new ModelInput
    {
        Need = "Fun",
        Percentage = 14F,
    }
};

In [1]:
Predictions(sampleData);